In [21]:
# -*- coding:utf-8 -*-
# !/usr/bin/python
# Author: Selvaria
# 基于实际图片数据增强后的卷积模型

import pandas
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

df_data = pandas.read_csv(r'D:\data\Kaggle\MNIST\train.csv')
train_dir = 'D:/data/Kaggle/MNIST/picture/train'
test_dir = 'D:/data/Kaggle/MNIST/picture/test'

train_datagen = ImageDataGenerator(
    rescale = 1/255, #对图片的每个像素值均乘上这个放缩因子，把像素值放缩到0和1之间有利于模型的收敛
    rotation_range = 20, #角度值，0-180.表示图像随机旋转的角度范围
    width_shift_range = 0.1, #平移比例，下同
    height_shift_range = 0.1,
    shear_range = 0.2, #随机错切变换角度
    zoom_range = 0.2, #随机缩放比例
    horizontal_flip = False, #随机将一半图像水平翻转，主要用于真实世界的图像（即没有水平不对称的假设前提下）
    #zca_whitening=True, #白化处理，只适用于mnist这种只有1条通道的黑白图像
    fill_mode='nearest' #填充新创建像素的方法
)

test_datagen = ImageDataGenerator(rescale=1/255) #注意验证集的数据不能增强


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (28, 28),
    batch_size = 200, #每批量样本大小
    class_mode = 'categorical' #因为使用了categorical_crossentropy损失函数，所以用二进制标签
)

test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size = (28, 28),
    batch_size = 20,
    class_mode = 'categorical' 
)

Found 40000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [23]:
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras import models

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'same',activation ='relu', input_shape = (28,28,3)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'same', activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 28, 28, 32)        2432      
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 7, 7, 64)          0         
__________

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch = 200, #40000/200
    epochs = 30, #增加了训练批次
    validation_data = test_generator, 
    validation_steps = 100 #2000/20
)

Epoch 1/30
200/200 [==============================] - 167s 834ms/step - loss: 1.7555 - acc: 0.3552 - val_loss: 0.2591 - val_acc: 0.9225
Epoch 2/30
200/200 [==============================] - 171s 853ms/step - loss: 0.4123 - acc: 0.8731 - val_loss: 0.1206 - val_acc: 0.9660
Epoch 3/30
200/200 [==============================] - 159s 797ms/step - loss: 0.2621 - acc: 0.9201 - val_loss: 0.0964 - val_acc: 0.9720
Epoch 4/30
200/200 [==============================] - 165s 826ms/step - loss: 0.2103 - acc: 0.9352 - val_loss: 0.0801 - val_acc: 0.9765
Epoch 5/30
200/200 [==============================] - 168s 839ms/step - loss: 0.1823 - acc: 0.9452 - val_loss: 0.0766 - val_acc: 0.9740
Epoch 6/30
200/200 [==============================] - 160s 801ms/step - loss: 0.1630 - acc: 0.9493 - val_loss: 0.0658 - val_acc: 0.9795
Epoch 7/30
200/200 [==============================] - 162s 810ms/step - loss: 0.1502 - acc: 0.9538 - val_loss: 0.0554 - val_acc: 0.9800
Epoch 8/30
200/200 [============================

In [15]:
print(model.evaluate_generator(test_generator, steps = 30))

[0.16970274890247766, 0.9624999910593033]
